In [3]:
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
# OPENAI_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv()

True

In [1]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain.prompts import PromptTemplate

In [4]:
# CSV 파일 불러오기
df = pd.read_csv("./data/storageInfo.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   category       60 non-null     object
 1   name           60 non-null     object
 2   storageDays    60 non-null     int64 
 3   storageDesc    60 non-null     object
 4   storageMethod  60 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.5+ KB


In [5]:
df.head()

,category,name,storageDays,storageDesc,storageMethod
0,기타,두부,7,5~7일,물에 담가 밀폐용기에 넣어 냉장 보관. 매일 물을 갈아주면 더 오래 보관 가능합니다.
1,야채,콩나물,5,3~5일,씻지 않은 상태로 물기를 제거하고 밀폐용기에 담아 냉장 보관하세요. 빛을 차단하면 ...
2,야채,숙주나물,4,2~4일,"콩나물보다 쉽게 무르므로, 씻지 않고 밀폐용기에 담아 냉장 보관 후 가급적 빨리 소..."
3,야채,고구마,30,2~4주,10~15°C의 서늘하고 어두운 곳에 신문지로 감싸 보관하세요. (냉장 보관 시 맛...
4,야채,감자,30,2~4주,빛이 없는 서늘하고 통풍이 잘 되는 곳에 보관하세요. 사과와 함께 두면 싹 나는 것...


In [6]:
# PandasDataFrameOutputParser 설정
parser = PandasDataFrameOutputParser(dataframe=df)
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [7]:
df.tail()

,category,name,storageDays,storageDesc,storageMethod
55,과일,레몬,28,3~4주,낱개로 랩이나 비닐 팩에 싸서 냉장 보관하면 수분 증발을 막을 수 있습니다.
56,과일,망고,5,후숙 후 3~5일,검은 반점이 생길 때까지 실온에서 후숙시킨 후 냉장 보관하세요.
57,과일,체리,7,5~7일,씻지 않은 상태로 밀폐용기에 담아 냉장 보관하세요.
58,과일,석류,21,2~3주,통째로 서늘한 실온이나 냉장 보관하세요.
59,과일,아보카도,5,후숙 후 3~5일,"단단하면 실온에서 후숙시키고, 익은 후에는 냉장 보관하세요."


In [8]:
# parser내부에 저장된 df를 확인하는 방법
print(parser.dataframe.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   category       60 non-null     object
 1   name           60 non-null     object
 2   storageDays    60 non-null     int64 
 3   storageDesc    60 non-null     object
 4   storageMethod  60 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.5+ KB
None


In [ ]:
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM 초기화
# llm = ChatOpenAI(model="gpt-4o", temperature=0)
# llm = ChatOllama(model="gemma3:27b", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemma-3-27b-it")

# 프롬프트 템플릿 생성
prompt = PromptTemplate(
    template="""Answer the user query.
{format_instructions}

User Query: {query}

Remember to format your response exactly as specified in the instructions.
For example:
- To get a column: "column:column_name"
- To get a row: "row:row_number"
- To get mean: "mean:column_name"
- To count values: "value_counts:column_name"
""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [14]:
# 분석 쿼리 실행 함수
def analyze_data(query):
    """데이터프레임에 대한 쿼리를 실행하는 함수"""
    chain = prompt | llm | parser
    try:
        result = chain.invoke({"query": query})
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None

In [15]:
# 다양한 분석 예제
print("\n=== 데이터 분석 예제 ===")

# 1. 특정 컬럼 조회
print("\n1. category 컬럼 조회:")
query1 = "Get the category column"
result1 = analyze_data(query1)
if result1:
    print(f"카테고리 종류: {set(result1['category'])}")
    print(f"총 데이터 수: {len(result1['category'])}")


=== 데이터 분석 예제 ===

1. category 컬럼 조회:
카테고리 종류: {'과일', '야채', '기타'}
총 데이터 수: 60


In [16]:
# 2. storageDays 컬럼 조회
print("\n2. storageDays 컬럼 조회:")
query2 = "Get the storageDays column"
result2 = analyze_data(query2)
if result2:
    storage_days = pd.Series(result2["storageDays"])
    print(f"평균 보관일수: {storage_days.mean():.1f}일")
    print(f"최대 보관일수: {storage_days.max()}일")
    print(f"최소 보관일수: {storage_days.min()}일")


2. storageDays 컬럼 조회:
평균 보관일수: 15.9일
최대 보관일수: 180일
최소 보관일수: 3일


In [17]:
# 3. 첫 번째 행 조회
print("\n3. 첫 번째 행 데이터:")
query3 = "Retrieve the first row (row 0)"
result3 = analyze_data(query3)
if result3:
    for key, value in result3["0"].items():
        print(f"  {key}: {value}")


3. 첫 번째 행 데이터:
  category: 기타
  name: 두부
  storageDays: 7
  storageDesc: 5~7일
  storageMethod: 물에 담가 밀폐용기에 넣어 냉장 보관. 매일 물을 갈아주면 더 오래 보관 가능합니다.


In [18]:
# 4. 특정 행 범위의 name 컬럼
print("\n4. 처음 5개 식품명:")
query4 = "Get the name column for rows 0 to 4"
result4 = analyze_data(query4)
if result4:
    for idx, name in result4["name"].items():
        if int(idx) < 5:
            print(f"  {idx}: {name}")


4. 처음 5개 식품명:
  0: 두부
  1: 콩나물
  2: 숙주나물
  3: 고구마
  4: 감자


In [ ]:
# 직접 pandas로 분석 (비교용)
print("\n=== Pandas 직접 분석 ===")

# 카테고리별 개수
print("\n카테고리별 식품 개수:")
category_counts = df["category"].value_counts()
print(category_counts)

# 카테고리별 평균 보관일수
print("\n카테고리별 평균 보관일수:")
category_avg = df.groupby("category")["storageDays"].mean().round(1)
print(category_avg)

# 보관일수 30일 이상 식품
print("\n보관일수 30일 이상인 식품:")
long_storage = df[df["storageDays"] >= 30][["name", "storageDays", "category"]]
print(long_storage.head(10))

# 기술 통계량
print("\n=== 기술 통계량 ===")
print(df["storageDays"].describe())


=== Pandas 직접 분석 ===

카테고리별 식품 개수:
category
야채    33
과일    24
기타     3
Name: count, dtype: int64

카테고리별 평균 보관일수:
category
과일    11.7
기타    67.0
야채    14.4
Name: storageDays, dtype: float64

보관일수 30일 이상인 식품:
   name  storageDays category
3   고구마           30       야채
4    감자           30       야채
5    양파           60       야채
6    마늘           60       야채
50   곶감          180       기타

=== 기술 통계량 ===
count     60.000000
mean      15.950000
std       24.316016
min        3.000000
25%        7.000000
50%       10.000000
75%       15.750000
max      180.000000
Name: storageDays, dtype: float64
